# SCD2 Dimension

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/02_core_patterns/scd2_dimension/playbook.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic/blob/main/examples/02_core_patterns/scd2_dimension/playbook.ipynb)

## Business Scenario

Customer attributes change over time. Reporting and compliance require you to know what was true on a specific date, not just the latest snapshot.

## Value Proposition

- Preserve full history of attribute changes
- Track active vs. expired records
- Support time-travel analytics

---

## Goals

1. Load multiple snapshots
2. Close out old rows
3. Produce an SCD2 table


In [ ]:
from pathlib import Path
import csv
import shutil
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "02_core_patterns" / "scd2_dimension" / "contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

output_dir = BASE / "output"
if output_dir.exists():
    shutil.rmtree(output_dir)

def read_csv_rows(path, limit=5):
    rows = []
    with open(path, newline="", encoding="utf-8") as handle:
        reader = csv.DictReader(handle)
        for idx, row in enumerate(reader):
            rows.append(row)
            if idx + 1 >= limit:
                break
    return rows

def preview_rows(rows, title=None):
    if title:
        print(title)
    if not rows:
        print("<no rows>")
        return
    for row in rows:
        print(row)


In [ ]:
processor = DataProcessor(contract=contract_path)

snapshot1_path = BASE / "data" / "dim_customers_snapshot1.csv"
result1 = processor.run_source(snapshot1_path)
processor.materialize(result1.good, result1.bad)

output_path = BASE / "output" / "gold_dim_customers" / "data.csv"
preview_rows(read_csv_rows(output_path, limit=5), "SCD2 TABLE AFTER SNAPSHOT 1:")


In [ ]:
snapshot2_path = BASE / "data" / "dim_customers_snapshot2.csv"
result2 = processor.run_source(snapshot2_path)
processor.materialize(result2.good, result2.bad)

output_path = BASE / "output" / "gold_dim_customers" / "data.csv"
preview_rows(read_csv_rows(output_path, limit=5), "SCD2 TABLE AFTER SNAPSHOT 2:")
